In [1]:
import os
import torch

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


emotions_dict = {
    '0': 'neutral',
    '1': 'calm',
    '2': 'happy',
    '3': 'sad',
    '4': 'angry',
    '5': 'fearful',
    '6': 'disgust',
    '7': 'surprised'
}


sub_name = 'Deep-Net'
data_name = 'RAVDESS'

minibatch = 64
num_epochs = 1000
EARLY_CNT = 99999

learning_rate = 1e-8

In [2]:
from cv2 import transform
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
from einops import rearrange
import torch.nn.functional as F
from torchinfo import summary
from tqdm.auto import tqdm
import math

In [3]:
def get_padding(kernel_size: int, stride: int, dilation: int) -> int:
    if stride > 1 and dilation > 1:
        raise ValueError('Only stride OR dilation may be greater than 1')
    return (kernel_size // 2) * dilation

In [4]:
class Model(nn.Module):
    """Some Information about Model"""
    def __init__(self, num_emotions):
        super(Model, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 10, kernel_size=[9,1], padding=(4,0)),
            nn.ReLU(),
            nn.Conv2d(10, 10, kernel_size=[5,1], padding=(2,0)),
            nn.ReLU(),
            nn.Conv2d(10, 10, kernel_size=[3,1], padding=(1,0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=[3,1], stride=1),
            nn.BatchNorm2d(10)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(10, 40, kernel_size=[3,1], padding=(1,0)),
            nn.ReLU(),
            nn.Conv2d(40, 40, kernel_size=[3,1], padding=(1,0)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=[2,1]),
            nn.BatchNorm2d(40)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(40, 80, kernel_size=[10,1], padding=(5,0)),
            nn.ReLU(),
            nn.Conv2d(80, 80, kernel_size=[1,1]),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=[2,1]),
            nn.BatchNorm2d(80)
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(80, 80, kernel_size=[1,1]),
            nn.ReLU()
        )
        
        self.fc1 = nn.Linear(675840, 128)
        self.fc2 = nn.Linear(128, num_emotions)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        embd = self.conv1(x)
        embd = self.conv2(embd)
        embd = self.conv3(embd)
        embd = self.conv4(embd)
        
        embd = torch.flatten(embd, start_dim=1)
        
        out = self.fc1(embd)
        output_logits = self.fc2(out)
        output_softmax = self.softmax(output_logits)

        return output_logits, output_softmax

In [5]:
def make_train_step(model, criterion, optimizer):

    # define the training step of the training phase
    def train_step(X, Y):

        # forward pass
        output_logits, output_softmax = model(X)
        predictions = torch.argmax(output_softmax, dim=1)
        accuracy = torch.sum(Y == predictions)/float(len(Y))

        # compute loss on logits because nn.CrossEntropyLoss implements log softmax
        loss = criterion(output_logits, Y)

        # compute gradients for the optimizer to use
        loss.backward()

        # update network parameters based on gradient stored (by calling loss.backward())
        optimizer.step()

        # zero out gradients for next pass
        # pytorch accumulates gradients from backwards passes (convenient for RNNs)
        optimizer.zero_grad()

        return loss.item(), accuracy*100
    return train_step

In [6]:
def make_validate_fnc(model, criterion):
    def validate(X, Y):

        # don't want to update any network parameters on validation passes: don't need gradient
        # wrap in torch.no_grad to save memory and compute in validation phase:
        with torch.no_grad():

            # set model to validation phase i.e. turn off dropout and batchnorm layers
            model.eval()
            
            # get the model's predictions on the validation set
            output_logits, output_softmax = model(X)
            predictions = torch.argmax(output_softmax, dim=1)

            # calculate the mean accuracy over the entire validation set
            accuracy = torch.sum(Y == predictions)/float(len(Y))

            # compute error from logits (nn.crossentropy implements softmax)
            loss = criterion(output_logits, Y)

        return loss.item(), accuracy*100, predictions
    return validate

In [7]:
def make_save_checkpoint():
    def save_checkpoint(optimizer, model, epoch, filename):
        checkpoint_dict = {
            'optimizer': optimizer.state_dict(),
            'model': model.state_dict(),
            'epoch': epoch
        }
        torch.save(checkpoint_dict, filename)
    return save_checkpoint


def load_checkpoint(optimizer, model, filename):
    checkpoint_dict = torch.load(filename)
    epoch = checkpoint_dict['epoch']
    model.load_state_dict(checkpoint_dict['model'])
    if optimizer is not None:
        optimizer.load_state_dict(checkpoint_dict['optimizer'])
    return epoch

In [8]:
def criterion(predictions, targets):
    return nn.CrossEntropyLoss()(input=predictions, target=targets)

In [9]:
from einops import rearrange

def train(model, num_epochs, X_train, Y_train, X_valid, Y_valid, pkl_name, fold, optimizer):
    
    # instantiate lists to hold scalar performance metrics to plot later
    train_losses = []
    valid_losses = []

    # get training set size to calculate # iterations and minibatch indices
    train_size = X_train.shape[0]

    # instantiate the checkpoint save function
    save_checkpoint = make_save_checkpoint()

    # instantiate the training step function
    train_step = make_train_step(model, criterion, optimizer=optimizer)
    # instantiate the validation loop function
    validate = make_validate_fnc(model, criterion)

    cur_valid_loss = 0x3f3f3f
    early_stop_cnt = 0
    
    for epoch in tqdm(range(num_epochs)):

        if early_stop_cnt >= EARLY_CNT:
            break
        
        # set model to train phase
        model.train()

        # shuffle entire training set in each epoch to randomize minibatch order
        train_indices = np.random.permutation(train_size)

        # shuffle the training set for each epoch:
        X_train = X_train[train_indices, :, :]
        Y_train = Y_train[train_indices]

        # instantiate scalar values to keep track of progress after each epoch so we can stop training when appropriate
        epoch_acc = 0
        epoch_loss = 0
        num_iterations = int(train_size / minibatch)

        # create a loop for each minibatch of 32 samples:
        for i in range(num_iterations):

            # we have to track and update minibatch position for the current minibatch
            # if we take a random batch position from a set, we almost certainly will skip some of the data in that set
            # track minibatch position based on iteration number:
            batch_start = i * minibatch
            # ensure we don't go out of the bounds of our training set:
            batch_end = min(batch_start + minibatch, train_size)
            # ensure we don't have an index error
            actual_batch_size = batch_end-batch_start

            # get training minibatch with all channnels and 2D feature dims
            X = X_train[batch_start:batch_end, :, :]
            # get training minibatch labels
            Y = Y_train[batch_start:batch_end]
            
            # instantiate training tensors
            X_tensor = torch.tensor(X, device=device).float()
            Y_tensor = torch.tensor(Y, dtype=torch.long, device=device)
            
            # Pass input tensors thru 1 training step (fwd+backwards pass)
            loss, acc = train_step(X_tensor, Y_tensor)

            # aggregate batch accuracy to measure progress of entire epoch
            epoch_acc += acc * actual_batch_size / train_size
            epoch_loss += loss * actual_batch_size / train_size

            # keep track of the iteration to see if the model's too slow
            print('\r'+f'Epoch {epoch}: iteration {i}/{num_iterations}', end='')

        X_valid_tensor = torch.tensor(X_valid, device=device).float()
        Y_valid_tensor = torch.tensor(Y_valid, dtype=torch.long, device=device)
        
        # calculate validation metrics to keep track of progress; don't need predictions now
        valid_loss, valid_acc, _ = validate(X_valid_tensor, Y_valid_tensor)
        
        # accumulate scalar performance metrics at each epoch to track and plot later
        train_losses.append(epoch_loss)
        valid_losses.append(valid_loss)
        
        if cur_valid_loss > valid_loss:
            save_checkpoint(optimizer, model, epoch, pkl_name)
            cur_valid_loss = valid_loss
        else:
            early_stop_cnt += 1

        # keep track of each epoch's progress
        print(f'\nEpoch {epoch} --- loss:{epoch_loss:.3f}, Epoch accuracy:{epoch_acc:.2f}%, Validation loss:{valid_loss:.3f}, Validation accuracy:{valid_acc:.2f}%')

        
    print(f"\n\n[*] done !")
    print(f'[*] Best training loss - {min(train_losses)}')
    print(f'[*] Best validation loss - {min(valid_losses)}')
    
    return train_losses, valid_losses

In [10]:
import csv
import numpy as np
from PIL import Image
from skimage.io import imread
from skimage.transform import resize

def get_fold_dataset(npy_name):
    with open(npy_name, 'rb') as f:
        X_train = np.load(f)
        y_train = np.load(f)
        X_test = np.load(f)
        y_test = np.load(f)
    
    return X_train, y_train, X_test, y_test

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

train_losses, valid_losses = [], []

print(f'[*] model training - {sub_name}')

fold_path = 'Deep-Net dataset'

for fold in range(5):
    cur_fold_path = os.path.join(fold_path, 'fold'+str(fold)+'.npy')
    X_train, y_train, X_test, y_test = get_fold_dataset(cur_fold_path)
    
    pkl_name = f'./model/{data_name}-{sub_name}-{fold}.pkl'
    model = Model(num_emotions=len(emotions_dict)).to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0., amsgrad=True)
    
    train_losses, valid_losses = train(model, num_epochs, X_train, y_train, X_test, y_test, pkl_name, fold, optimizer)
    
    plt.title('Loss Curve')
    plt.ylabel('Loss', fontsize=16)
    plt.xlabel('Epoch', fontsize=16)
    plt.plot(train_losses[:],'b')
    plt.plot(valid_losses[:],'r')
    plt.legend(['Training loss','Validation loss'])
    plt.show()
    
    break

[*] model training - Deep-Net


  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 0: iteration 16/17
Epoch 0 --- loss:1.988, Epoch accuracy:12.81%, Validation loss:2.078, Validation accuracy:13.33%
Epoch 1: iteration 16/17
Epoch 1 --- loss:1.984, Epoch accuracy:12.89%, Validation loss:2.073, Validation accuracy:13.33%
Epoch 2: iteration 16/17
Epoch 2 --- loss:1.981, Epoch accuracy:12.72%, Validation loss:2.070, Validation accuracy:14.00%
Epoch 3: iteration 16/17
Epoch 3 --- loss:1.975, Epoch accuracy:13.68%, Validation loss:2.066, Validation accuracy:10.67%
Epoch 4: iteration 16/17
Epoch 4 --- loss:1.973, Epoch accuracy:13.95%, Validation loss:2.064, Validation accuracy:14.00%
Epoch 5: iteration 16/17
Epoch 5 --- loss:1.969, Epoch accuracy:13.51%, Validation loss:2.061, Validation accuracy:12.00%
Epoch 6: iteration 16/17
Epoch 6 --- loss:1.967, Epoch accuracy:12.98%, Validation loss:2.058, Validation accuracy:12.33%
Epoch 7: iteration 16/17
Epoch 7 --- loss:1.965, Epoch accuracy:13.86%, Validation loss:2.056, Validation accuracy:12.67%
Epoch 8: iteration 16/17

Epoch 67: iteration 16/17
Epoch 67 --- loss:1.858, Epoch accuracy:37.28%, Validation loss:1.980, Validation accuracy:30.00%
Epoch 68: iteration 16/17
Epoch 68 --- loss:1.858, Epoch accuracy:37.72%, Validation loss:1.979, Validation accuracy:30.67%
Epoch 69: iteration 16/17
Epoch 69 --- loss:1.856, Epoch accuracy:37.98%, Validation loss:1.979, Validation accuracy:31.00%
Epoch 70: iteration 16/17
Epoch 70 --- loss:1.856, Epoch accuracy:37.89%, Validation loss:1.978, Validation accuracy:31.00%
Epoch 71: iteration 16/17
Epoch 71 --- loss:1.852, Epoch accuracy:37.63%, Validation loss:1.976, Validation accuracy:30.00%
Epoch 72: iteration 16/17
Epoch 72 --- loss:1.851, Epoch accuracy:37.37%, Validation loss:1.975, Validation accuracy:29.33%
Epoch 73: iteration 16/17
Epoch 73 --- loss:1.852, Epoch accuracy:37.11%, Validation loss:1.974, Validation accuracy:30.67%
Epoch 74: iteration 16/17
Epoch 74 --- loss:1.849, Epoch accuracy:37.37%, Validation loss:1.973, Validation accuracy:30.00%
Epoch 75

Epoch 133: iteration 16/17
Epoch 133 --- loss:1.767, Epoch accuracy:42.11%, Validation loss:1.917, Validation accuracy:32.67%
Epoch 134: iteration 16/17
Epoch 134 --- loss:1.765, Epoch accuracy:42.63%, Validation loss:1.917, Validation accuracy:31.33%
Epoch 135: iteration 16/17
Epoch 135 --- loss:1.767, Epoch accuracy:41.84%, Validation loss:1.916, Validation accuracy:31.33%
Epoch 136: iteration 16/17
Epoch 136 --- loss:1.764, Epoch accuracy:42.19%, Validation loss:1.915, Validation accuracy:32.00%
Epoch 137: iteration 16/17
Epoch 137 --- loss:1.761, Epoch accuracy:42.28%, Validation loss:1.915, Validation accuracy:32.00%
Epoch 138: iteration 16/17
Epoch 138 --- loss:1.760, Epoch accuracy:42.54%, Validation loss:1.913, Validation accuracy:32.33%
Epoch 139: iteration 16/17
Epoch 139 --- loss:1.760, Epoch accuracy:41.58%, Validation loss:1.912, Validation accuracy:32.00%
Epoch 140: iteration 16/17
Epoch 140 --- loss:1.757, Epoch accuracy:42.37%, Validation loss:1.912, Validation accuracy

Epoch 198: iteration 16/17
Epoch 198 --- loss:1.691, Epoch accuracy:44.91%, Validation loss:1.868, Validation accuracy:34.00%
Epoch 199: iteration 16/17
Epoch 199 --- loss:1.688, Epoch accuracy:44.91%, Validation loss:1.868, Validation accuracy:33.67%
Epoch 200: iteration 16/17
Epoch 200 --- loss:1.689, Epoch accuracy:45.70%, Validation loss:1.867, Validation accuracy:33.33%
Epoch 201: iteration 16/17
Epoch 201 --- loss:1.689, Epoch accuracy:45.79%, Validation loss:1.866, Validation accuracy:33.33%
Epoch 202: iteration 16/17
Epoch 202 --- loss:1.687, Epoch accuracy:45.26%, Validation loss:1.865, Validation accuracy:33.67%
Epoch 203: iteration 16/17
Epoch 203 --- loss:1.685, Epoch accuracy:45.44%, Validation loss:1.864, Validation accuracy:33.67%
Epoch 204: iteration 16/17
Epoch 204 --- loss:1.687, Epoch accuracy:45.44%, Validation loss:1.864, Validation accuracy:33.67%
Epoch 205: iteration 16/17
Epoch 205 --- loss:1.686, Epoch accuracy:45.26%, Validation loss:1.863, Validation accuracy

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

device = 'cuda'
model = Model(num_emotions=len(emotions_dict)).to('cuda')

test_accs = []

for fold in range(5):
    pkl_name = f'./model/{data_name}-{sub_name}-{fold}.pkl'
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0., amsgrad=True)
    
    load_checkpoint(optimizer, model, pkl_name)
    
    validate = make_validate_fnc(model, criterion)
    
    cur_fold_path = os.path.join(fold_path, 'fold'+str(fold)+'.npy')
    X_train, y_train, X_test, y_test = get_fold_dataset(cur_fold_path)
    
    X_test_tensor = torch.tensor(X_test,device=device).float()
    # X_test_tensor = X_test_tensor.permute(0, 3, 1, 2)
    y_test_tensor = torch.tensor(y_test,dtype=torch.long,device=device)
    
    test_loss, test_acc, predicted_emotions = validate(X_test_tensor, y_test_tensor)
    print(f'[*] {fold} Test accuracy is {test_acc:.2f}%')
    test_accs.append(test_acc)
    
    predicted_emotions = predicted_emotions.cpu().numpy()
    # use labels from test set
    emotions_groundtruth = y_test_tensor.cpu().numpy()
    
    # build confusion matrix and normalized confusion matrix
    conf_matrix = confusion_matrix(emotions_groundtruth, predicted_emotions)
    conf_matrix_norm = confusion_matrix(emotions_groundtruth, predicted_emotions,normalize='true')

    # set labels for matrix axes from emotions
    emotion_names = [emotion for emotion in emotions_dict.values()]

    # make a confusion matrix with labels using a DataFrame
    confmatrix_df = pd.DataFrame(conf_matrix, index=emotion_names, columns=emotion_names)
    confmatrix_df_norm = pd.DataFrame(conf_matrix_norm, index=emotion_names, columns=emotion_names)

    # plot confusion matrices
    plt.figure(figsize=(16,6))
    sn.set(font_scale=1.8) # emotion label and title size
    plt.subplot(1,2,1)
    plt.title('Confusion Matrix')
    sn.heatmap(confmatrix_df, annot=True, annot_kws={"size": 18}) #annot_kws is value font
    plt.subplot(1,2,2)
    plt.title('Normalized Confusion Matrix')
    sn.heatmap(confmatrix_df_norm, annot=True, annot_kws={"size": 13}) #annot_kws is value font

    plt.show()
    
    break

In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

print('[*] K-Fold training done !')
print(f'[*] Best fold acc : {max(test_accs):.2f}%')
print(f'[*] Worst fold acc : {min(test_accs):.2f}%')
print(f'[*] Average fold acc : {Average(test_accs):.2f}%')

In [ ]:
from sklearn.metrics import classification_report

device = torch.device('cpu')
model = model.to(device)

test_accs = []

for fold in range(5):
    pkl_name = f'./model/{data_name}-{sub_name}-{fold}.pkl'
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0., amsgrad=True)
    
    load_checkpoint(optimizer, model, pkl_name)
    
    validate = make_validate_fnc(model, criterion)
    
    cur_fold_path = os.path.join(fold_path, 'fold'+str(fold)+'.npy')
    X_train, y_train, X_test, y_test = get_fold_dataset(cur_fold_path)
    
    X_test_tensor = torch.tensor(X_test).float()
    # X_test_tensor = X_test_tensor.permute(0, 3, 1, 2)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    test_loss, test_acc, predicted_emotions = validate(X_test_tensor, y_test_tensor)
    print(f'[*] {fold} Test accuracy is {test_acc:.2f}%')
    test_accs.append(test_acc)
    
    predicted_emotions = predicted_emotions.cpu().numpy()
    # use labels from test set
    emotions_groundtruth = y_test_tensor

    print(classification_report(emotions_groundtruth, predicted_emotions))
    
    break